# Transformando uma imagem do GEE em um Dataframe utilizando Reduce Region, Numpy e Pandas

Neste notebook, utilizando o exemplo de extração de máscaras, iremos transformar uma imagem em um DataFrame (estrutura de dados do Pandas) utilizando a função do GEE chamada de Reduce Region com auxílio das bibliotecas Numpy e Pandas

Primeiramente, vamos importar as bibliotecas e inicializar o GEE:

In [4]:
# importação da bibliotecas
import ee
import pandas as pd
import numpy as np

# inicialização do GEE
ee.Initialize() 

Funções principais utilizadas por esse notebook (comentadas no notebook anterior):

In [5]:
# Função para aplicar à imagem vinda da coleção a máscara de água
def mascara_agua(imagem):
    qa = imagem.select('pixel_qa')
    return qa.bitwiseAnd(1 << 2).eq(0)

# Função para aplicar à imagem vinda da coléção a máscara de nuvem/sombra de nuvem
def mascara_nuvem(imagem):
    qa = imagem.select('pixel_qa')
    return qa.bitwiseAnd(1 << 3).eq(0) and (qa.bitwiseAnd(1 << 5).eq(0)) and (qa.bitwiseAnd(1 << 6).eq(0)) and (qa.bitwiseAnd(1 << 7).eq(0))

# função para aplicar as máscaras
def aplicar_mascaras(imagem):
    
    # criar uma imagem em branco/vazio para evitar problemas no fundo ao gerar um PNG
    # usamos valores dummies (neste caso, branco)
    vazio = ee.Image(99999)
    
    # máscara de água
    agua = vazio.updateMask(mascara_agua(imagem).Not()).rename('agua')
    
    # máscara de nuvem (criará uma imagem com apenas nuvens)
    # caso a imagem não tenha nuvens, ela ficará toda branca
    nuvem = vazio.updateMask(mascara_nuvem(imagem).Not()).rename('nuvem')
    
    # podemos ainda, ao contrário da linha anterior, REMOVER as nuvens
    # notem que retiramos a função .Not (negação)
    sem_nuvem = vazio.updateMask(mascara_nuvem(imagem)).rename('sem_nuvem')
    
    # aplicar o indice NDVI
    ndvi = imagem.expression('(nir - red) / (nir + red)',{'nir':imagem.select('B5'),'red':imagem.select('B4')}).rename('ndvi')
    
    # assim como fizemos para o NDVI, retornamos uma imagem com as novas bandas
    return imagem.addBands([ndvi,agua,nuvem,sem_nuvem])

# função para aplicar uma máscara em uma banda específica
# A mascará a ser aplicada 
def aplicar_mascara_banda(imagem, banda_mascara, banda_origem, band_destino):
    
    # Primeiramente, temos que aplicar a máscara desejada na banda de origem, que será nomeada para a banda de destino
    # Podemos, inclusive, sobscrever a banda de origem, sem problemas
    imagem_mascara = imagem.select(banda_origem).updateMask(imagem.select(banda_mascara)).rename(band_destino)
    
    # Depois, temos que criar uma imagem em branco que receberá a máscara, renomeando também para banda de destino
    imagem_mascara = ee.Image(99999).blend(imagem_mascara).rename(band_destino)
    
    # Retornar a imagem com a nova banda nomeada com a string da banda_destino
    return imagem.addBands([imagem_mascara])

Agora, vamos definir a geometria e as datas (baseada na Latitude e Longitude) da nossa área de estudo e consultá-la no GEE (mesmo do notebook anterior):

In [6]:
# Notem que foi criada uma coordenada (Latitude e Longitude) através de uma string, posteriormente repartida pelas virgulas
# Essa abordagem é importante para quando utilizarmos a linha da comando
coordenadas = "-48.53801472648439,-22.503806214013736,-48.270222978437516,-22.7281869567509"

# Aqui, usamos uma ferramenta do Python chamada de unpacking
x1,y1,x2,y2 = coordenadas.split(",")

# Criamos a geometria com base nas coordenadas 'quebradas' acima
geometria = geometry = ee.Geometry.Polygon(
        [[[float(x1),float(y2)],
          [float(x2),float(y2)],
          [float(x2),float(y1)],
          [float(x1),float(y1)],
          [float(x1),float(y2)]]])

# String de datas
datas = "2014-10-13,2014-10-14"

# Divisão das duas datas pela vírgula, novamente usando a técnica de unpacking
inicio,fim = datas.split(",")

# Consultando a coleção com base na área de estudo e datas selecionadas
colecao = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(geometria).filterDate(inicio,fim).filterMetadata('CLOUD_COVER','less_than', 30)

# aplicar a função 'aplicar_mascaras' em todas as imagens (irá adicionar as bandas 'agua', 'nuvem', 'sem_nuvem' nas imagens):
colecao = colecao.map(aplicar_mascaras)

# extraindo a imagem mediana da coleção
imagem = colecao.median()

Agora, vamos aplicar as máscaras individualmente na banda NDVI:

In [7]:
# Aplicamos as três máscaras individualmente na banda NDVI
# A função irá adicionar as já mencionadas bandas de origem a medida que for sendo executada, linha a linha
imagem = aplicar_mascara_banda(imagem, 'agua', 'ndvi', 'ndvi_agua')
imagem = aplicar_mascara_banda(imagem, 'nuvem', 'ndvi', 'ndvi_nuvem')
imagem = aplicar_mascara_banda(imagem, 'sem_nuvem', 'ndvi', 'ndvi_sem_nuvem')
imagem = aplicar_mascara_banda(imagem, 'agua', 'ndvi_sem_nuvem', 'ndvi_agua_sem_nuvem')

# Depois, cortamos a imagem
# scale = escala do sensor. No caso do Landsat-8/OLI são 30 metros
imagem_corte = imagem.clipToBoundsAndScale(geometry=geometria,scale=30)

Introduzimos aqui uma nova função, responsável por extrair as informações de coordenadas e pixeis de uma ou mais bandas, de acordo com o seu parâmetro 'bandas':

In [10]:
# Extrair as coordenadas e valores dos pixels de uma imagem
def extrair_latitude_longitude_pixel(imagem, geometria, bandas):
  
    # Inicialmente, devemos extrair as coordenadas por pixel da imagem
    # O GEE faz essa operação adicoinando uma banda com essas novas informações,
    # que extrairemos abaixo
    imagem = imagem.addBands(ee.Image.pixelLonLat())
    
    # Extraindo efetivament as coordenadas nas bandas recém criadas (latitude e longitude)
    # Nesta parte, é utilizado o que citamos como reducer (verifiar na documentação do GEE), mas ele permite que sejam feitas operações com uma imagem como: reduzi-la, modificar sua escala, etc.
    # ainda, os atributos utilizados são: geometry (geometria, mesma utilizada em outros exemplos), scale (escala do sensor, 30 metros no caso do Landsat), bestEffort (garante que a imagem terá a melhor escala possível, caso a definida seja muito grande para processamento)
    coordenadas = imagem.select(['longitude', 'latitude']+bandas).reduceRegion(reducer=ee.Reducer.toList(),geometry=geometria,scale=30,bestEffort=True)
    
    # ponteiro para incluir os valores dos pixeis de cada banda, já criando uma Numpy Array
    # o FOR abaixo irá percorrer cada banda que foi definida no parâmetro da função para extrair seus valores, um a um
    # As funções ee.List e getInfo() permitem transformar os pixeis em lista e depois extraí-los, respectivamente
    bandas_valores = []
    for banda in bandas:
        
        # adiciona pixel por pixel, em cada uma das bandas desejadas
        # transforma o valor do pixel em float para evitar erros de processamento futuros
        bandas_valores.append(np.array(ee.List(coordenadas.get(banda)).getInfo()).astype(float))
    
    
    # Retorna no forma de Numpy Array os dados separados pelas colunas [0,1,2..N BANDAS] sendo LATITUDE, LONGITUDE e VALOR DO PIXEL (POR BANDA...N BANDAS)
    return np.array(ee.List(coordenadas.get('latitude')).getInfo()).astype(float), np.array(ee.List(coordenadas.get('longitude')).getInfo()).astype(float), bandas_valores

Com a função acima criada, vamos utilizá-la abaixo:

In [24]:
# Com a imagem, iremos extrair as coordenadas e o indice NDVI desejados, já com a máscara de água aplicada
# Usamos aqui novamente a técnica de unpacking (atentar para ordem: long,lat,indices)
longitudes, latitudes, indices = extrair_latitude_longitude_pixel(imagem_corte, geometria, ['ndvi_agua_sem_nuvem'])

# Mostra o número de pixeis das arrays extraídas da imagem
print(len(latitudes))
print(len(longitudes))
print(len(indices[0])) # neste caso 0 porque iremos trabalhar com apenas um indice (NDVI), mas a função aceita também array de bandas

827008
827008
827008


Finalizamos este notebook apresentando a criação de um DataFrame Pandas com os dados extraídos da imagem. Embora seja possível trabalhar tranquilamente com o Numpy nos dados desses arrays, o Pandas oferece suporte à gráficos integrados do Matplotlib, modelos de consulta de dados, agrupamento e outras ferramentas que facilitam a vida do cientista.

In [26]:
# Criando o Dataframe com as colunas desejadas
# Adicionamos os dados extraídos da imagem neste Dataframe, respeitando as colunas
# Usamos aqui a função zip, pra juntar as colunas, mantendo a relação de linhas (L1_COL1,L1_COL2,L1_COl3...L1_COLN)
# Existem outras maneiras de atingir o mesmo resultado
df = pd.DataFrame(columns=['latitude','longitude', 'ndvi'],data=list(zip(latitudes,longitudes,indices[0])))
print(df.head())

    latitude  longitude     ndvi
0 -48.537996 -22.727961  99999.0
1 -48.537727 -22.727961  99999.0
2 -48.537457 -22.727961  99999.0
3 -48.537188 -22.727961  99999.0
4 -48.536918 -22.727961  99999.0


### Atenção: a coluna ndvi apresenta valores dummies para as áreas fora da água por conta da utilização do valor dummy '99999' nos pixels que ocorrem fora do corpo d'água. Isto é feito no momento da extração da máscara. Podemos modificar esse valor para qualquer outro, desde que não colida com outros valores presentes nos dados observados.